# Collect training data using CDL 

* This is a simple jupyter notebook to collect random training data using CDL imagery. 

* Manual data collection is tedious, so we generate a training dataset by randomly picking labels from CDL

In [3]:
import rasterio as rio
import geop#### How to read geotiff files?andas as gpd
import numpy as np
import pandas as pd
np.random.seed(100)

### Check the counts of number of pixels for each class in original CDL file
* This code uses a clipped raster containing class labels defined by CDL for a region around Asheville
* The raster has been projected to EPSG:4326 
* We are using rasterio to read geotiff files.
* Read CDL (Crop Land Data Layer) documentation to identify what each class number means (for eg. class 111 = water)

In [5]:
# connect to the file
data_src = rio.open('./CDL_2017.tif')
# remove any additional dimensions
# data_src.read(i) reads band number i
# CDL has only one band
data = data_src.read(1)
all_labels = np.unique(data)
for l in all_labels:
    print(f'Class: {l}, # Vals = {np.sum(data==l)}')

Class: 0, # Vals = 5825
Class: 1, # Vals = 9588
Class: 5, # Vals = 675
Class: 26, # Vals = 4
Class: 36, # Vals = 97
Class: 37, # Vals = 58790
Class: 42, # Vals = 2
Class: 44, # Vals = 4
Class: 54, # Vals = 1677
Class: 59, # Vals = 829
Class: 61, # Vals = 3391
Class: 68, # Vals = 2275
Class: 70, # Vals = 11
Class: 111, # Vals = 7264
Class: 121, # Vals = 236274
Class: 122, # Vals = 55497
Class: 123, # Vals = 21846
Class: 124, # Vals = 7391
Class: 131, # Vals = 440
Class: 141, # Vals = 1175510
Class: 142, # Vals = 42322
Class: 143, # Vals = 1671
Class: 152, # Vals = 2500
Class: 176, # Vals = 124308
Class: 190, # Vals = 4
Class: 195, # Vals = 1
Class: 225, # Vals = 459
Class: 245, # Vals = 14
Class: 248, # Vals = 3


### Data sampling
For this exercise, let us assume we are considering 5 classes. CDL classes are much more descriptive, we are generalizing them for this exercise,

#### Class information:
* Class 0 - forest (includes data from cdl classes: 'forest', 'deciduous forest', 'evergreen forest', 'mixed forest')
* Class 1 - corn (includes data from cdl class 'corn')
* Class 2 - soy (includes data from cdl class 'soybeans')
* Class 3 - developed/urban (includes data from cdl classes 'developed/open space', 'developed/low intensity', 'developed/med intensity', 'developed/high intensity')
* Class 4 - water (includes data from cdl class 'open water')

### Where is the data picked from in the raster?
To avoid challenges associated with edges of a raster, we do not consider any data that occurs at the corners.

### How much data is picked?
A maximum of 500 points are picked per class. Please note that some classes may have less than 500 points (either because there aren't that many data points, or these points are at the borders and we ignored them)



In [6]:
n_points = 500
labels = ['forest', 'corn', 'soy', 'developed', 'water']
supersets = {0: [63, 141, 142, 143],
           1: [1],
            2: [5],
            3: [121, 122, 123, 124],
            4: [111]}
locs_dict = {}
for key in list(supersets.keys()):
    sub_labels = supersets[key]
    key_counts = 0
    sub_dict = {}
    sub_locs = None
    for idx, s_l in enumerate(sub_labels):
        sub_mask = data == s_l
        # set first 30 rows, cols and last 30 rows, cols to false so that it doesn't pick from corners
        sub_mask[0:30, :] = False
        sub_mask[-30:, :] = False
        sub_mask[:, 0:30] = False
        sub_mask[:, -30:] = False
        sub_mask_ind = np.argwhere(sub_mask)
        
        sub_mask_sub = sub_mask_ind[np.random.choice(sub_mask_ind.shape[0], min(sub_mask_ind.shape[0], int(n_points/len(sub_labels))), replace=False), :]
        key_counts += np.sum(data==s_l)
        if sub_locs is None:
            sub_locs = np.copy(sub_mask_sub)
        else:
            sub_locs = np.vstack((sub_locs, sub_mask_sub))
    locs_dict[key] = sub_locs
    
        
    #print(f'Actual counts = {key_counts}, counts using mask = {np.sum(label_mask)}, locs dict count = {locs_dict[key].shape}')    
        

### Generate a data frame with the lat-longs and class information 

In [7]:
landsat_src = rio.open('./L8_2017.tif')
pandas_df = pd.DataFrame()
for key in list(locs_dict.keys()):
    # first get lat long for all values in the numpy array
    lat_long_list = []
    curr_indices = locs_dict[key]
    print(f'Class {key} has {curr_indices.shape[0]} rows')
    for i in range(curr_indices.shape[0]):
        cdl_x, cdl_y = curr_indices[i,0], curr_indices[i, 1]
        cdl_long, cdl_lat = data_src.xy(cdl_x, cdl_y)
        new_df = pd.DataFrame({'Latitude': [cdl_lat], 'Longitude': [cdl_long], 'Class':key})
        pandas_df = pandas_df.append(new_df, ignore_index=True)
        

Class 0 has 375 rows
Class 1 has 500 rows
Class 2 has 500 rows
Class 3 has 500 rows
Class 4 has 500 rows


### Finally, convert the data frame to a shape file 
* We use geopandas to first convert the pandas data frame to a geodataframe with a point representing each location picked 
* We then save this geopandas file to disk

In [8]:
geopandas_df = gpd.GeoDataFrame(
    pandas_df, geometry=gpd.points_from_xy(pandas_df.Longitude, pandas_df.Latitude))

In [9]:
geopandas_df.crs = {'init' :'epsg:4326'}

In [10]:
geopandas_df

,Latitude,Longitude,Class,geometry
0,35.514769,-82.680451,0,POINT (-82.68045 35.51477)
1,35.753979,-82.520432,0,POINT (-82.52043 35.75398)
2,35.710635,-82.305661,0,POINT (-82.30566 35.71063)
3,35.512814,-82.413861,0,POINT (-82.41386 35.51281)
4,35.520636,-82.853181,0,POINT (-82.85318 35.52064)
...,...,...,...,...
2370,35.664357,-82.338903,4,POINT (-82.33890 35.66436)
2371,35.479572,-82.536076,4,POINT (-82.53608 35.47957)
2372,35.481528,-82.534446,4,POINT (-82.53445 35.48153)
2373,35.663379,-82.338903,4,POINT (-82.33890 35.66338)


In [11]:
geopandas_df.to_file('./GroundTruth.shp')